In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.pipeline import Pipeline
from functions import *
import matplotlib.pyplot as plt
import joblib
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.metrics import mean_squared_error, r2_score


In [3]:
dates = pd.read_csv('df_dates_test.csv')
metrics = pd.read_csv('df_metrics_test.csv')
opens = pd.read_csv('df_opens_test.csv')
pred = pd.read_csv('df_pred_test.csv')

In [4]:
dates.rename(columns={'Value':'Dates_test'}, inplace=True)
dates.drop('Unnamed: 0', axis=1, inplace=True)

opens.rename(columns={'Value':'Open_test'}, inplace=True)
opens.drop(['Ticker', 'Unnamed: 0'], axis=1, inplace=True)

pred.rename(columns={'Value':'Close_pred_test'}, inplace=True)
pred.drop(['Ticker', 'Unnamed: 0'], axis=1, inplace=True)


In [5]:
df = pd.concat([opens, dates, pred], axis=1)
df

,Open_test,Ticker,Dates_test,Close_pred_test
0,45.520000,AADR,2023-03-24,45.759169
1,46.040001,AADR,2023-03-27,46.681240
2,47.049999,AADR,2023-03-28,46.993237
3,47.410000,AADR,2023-03-29,47.445742
4,48.209999,AADR,2023-03-30,48.081131
...,...,...,...,...
382301,19.500000,ZHDG,2024-07-02,19.320602
382302,19.673000,ZHDG,2024-07-03,19.496974
382303,19.768999,ZHDG,2024-07-05,19.686856
382304,19.889999,ZHDG,2024-07-08,19.689868


In [6]:
tickers = df.groupby('Ticker').count()
print(tickers)
print(tickers.max())
print(tickers.min())

        Open_test  Dates_test  Close_pred_test
Ticker                                        
AADR          324         324              324
AAPB           91          91               91
AAPD           91          91               91
AAPY           31          31               31
AAXJ          324         324              324
...           ...         ...              ...
YANG          324         324              324
YOLO          259         259              259
YXI           324         324              324
YYY           324         324              324
ZHDG          147         147              147

[1485 rows x 3 columns]
Open_test          324
Dates_test         324
Close_pred_test    324
dtype: int64
Open_test          18
Dates_test         18
Close_pred_test    18
dtype: int64


In [7]:
df['Dates_test'] = pd.to_datetime(df['Dates_test'])

# Ordenar datos por Ticker y fecha
df = df.sort_values(by=['Ticker', 'Dates_test'])

df['Open_15d'] = df.groupby('Ticker')['Open_test'].shift(15).dropna()

df.dropna(inplace=True)

# Calcular las rentabilidades diarias
df['Rentabilidad_diaria'] = (df['Close_pred_test'] - df['Open_15d']) / df['Open_15d']

df_sorted = df.sort_values(by='Dates_test', ascending=False)
df_sorted

,Open_test,Ticker,Dates_test,Close_pred_test,Open_15d,Rentabilidad_diaria
382305,19.799999,ZHDG,2024-07-09,19.638636,19.440001,0.010218
321915,47.615002,SKOR,2024-07-09,47.635395,47.492085,0.003018
355666,45.137001,UTWY,2024-07-09,45.225888,45.729732,-0.011018
72769,31.184999,ECML,2024-07-09,31.578939,31.506001,0.002315
346829,21.530001,TUA,2024-07-09,21.508895,21.459628,0.002296
...,...,...,...,...,...,...
260520,63.869999,PBE,2023-04-17,64.078698,60.189999,0.064607
143221,110.250000,FXH,2023-04-17,110.319128,102.980003,0.071267
324534,54.549999,SMMD,2023-04-17,54.828612,51.680000,0.060925
143545,29.389999,FXI,2023-04-17,29.616670,28.620001,0.034824


In [8]:
ultima_fecha = df.groupby('Ticker')['Dates_test'].max().reset_index()
ultima_fecha

# Paso 2: Merge para obtener los datos de la última fecha
df_ultima_fecha = pd.merge(df, ultima_fecha, on=['Ticker', 'Dates_test'])
df_ultima_fecha.to_csv('etfs_ultima_fecha_15d.csv')

In [9]:
df_ultima_fecha

,Open_test,Ticker,Dates_test,Close_pred_test,Open_15d,Rentabilidad_diaria
0,62.860001,AADR,2024-07-09,62.992689,62.630001,0.005791
1,29.610001,AAPB,2024-07-09,28.857473,26.240000,0.099751
2,16.860001,AAPD,2024-07-09,19.411267,17.943572,0.081795
3,27.500000,AAPY,2024-07-09,27.133608,27.000000,0.004948
4,73.849998,AAXJ,2024-07-09,74.063437,71.669998,0.033395
...,...,...,...,...,...,...
1480,8.530000,YANG,2024-07-09,8.655882,8.600000,0.006498
1481,3.260000,YOLO,2024-07-09,3.154076,3.500000,-0.098835
1482,17.219999,YXI,2024-07-09,17.213201,17.299999,-0.005017
1483,12.010000,YYY,2024-07-09,12.098302,11.930100,0.014099


In [12]:
import pandas as pd
import numpy as np
from scipy.optimize import minimize

# Función para calcular la rentabilidad esperada de la cartera
def calcular_rentabilidad(pesos, rentabilidades):
    return np.dot(pesos, rentabilidades)

# Función para calcular la varianza de la cartera
def calcular_varianza(pesos, cov_matrix):
    return np.dot(pesos.T, np.dot(cov_matrix, pesos))

# Restricciones
restricciones = [
    {'type': 'eq', 'fun': lambda pesos: np.sum(pesos) - 1},  # Suma de pesos debe ser 1
    {'type': 'ineq', 'fun': lambda pesos: pesos},  # Pesos >= 0
    {'type': 'eq', 'fun': lambda pesos: np.sum(np.where(pesos > 0.8, 1, 0)) - 10}]  # Máximo 10 ETFs

# Función de optimización para maximizar la rentabilidad y minimizar la varianza
def funcion_objetivo(pesos):
    rentabilidad = calcular_rentabilidad(pesos, rentabilidad_esperada)
    varianza = calcular_varianza(pesos, rentabilidades_diarias_matrix)
    return -(rentabilidad - lambda_risk_aversion * varianza)  # Negativo porque minimize()

'''***************************'''

# Inicializar pesos uniformemente
pesos_iniciales = np.ones(len(df_ultima_fecha)) / len(df_ultima_fecha)
print('Pesos iniciales:', pesos_iniciales)
print('\n')

# La media de la rentabilidad de cada ETF se multiplica por los pesos equitativos asignados anteriormente
df_ultima_fecha['rentabilidad_ponderada'] = df_ultima_fecha['Rentabilidad_diaria'].values * pesos_iniciales
print('Rentabilidad ponderada:', rentabilidad_ponderada)
print('\n')

# Normalizar los pesos para que sumen 1
pesos_normalizados = df_ultima_fecha['rentabilidad_ponderada'] / df_ultima_fecha['rentabilidad_ponderada'].sum()
print('Pesos normalizados:', pesos_normalizados)
print('\n')

# Asegurarse de que la inversión esté diversificada en al menos 10 ETFs
# Ajustamos los pesos para cumplir con esta restricción
min_etfs = 10
etfs_seleccionados = pesos_normalizados.nlargest(min_etfs) # Seleccionamos los etf con mayores pesos (normalizados) para calcular ahora 

print('ETFs con mayor rentabilidad:', etfs_seleccionados)


Pesos iniciales: [0.0006734 0.0006734 0.0006734 ... 0.0006734 0.0006734 0.0006734]


Rentabilidad ponderada: [ 3.89963453e-06  6.71725758e-05  5.50808356e-05 ... -3.37860461e-06
  9.49426122e-06  6.88073899e-06]


Pesos normalizados: 0       0.000549
1       0.009459
2       0.007757
3       0.000469
4       0.003167
          ...   
1480    0.000616
1481   -0.009373
1482   -0.000476
1483    0.001337
1484    0.000969
Name: rentabilidad_ponderada, Length: 1485, dtype: float64


ETFs con mayor rentabilidad: 1341    0.085786
158     0.032968
1438    0.024008
113     0.018507
1342    0.017523
1340    0.016735
880     0.015196
1352    0.013418
1351    0.010663
571     0.010245
Name: rentabilidad_ponderada, dtype: float64


In [13]:
print("\nPesos Óptimos para cada ETF:")
for ticker, peso in pesos_normalizados.items():
    print(f"{ticker}: {peso:.2%}")


Pesos Óptimos para cada ETF:
0: 0.05%
1: 0.95%
2: 0.78%
3: 0.05%
4: 0.32%
5: -0.27%
6: 0.03%
7: 0.19%
8: 0.12%
9: -0.08%
10: -0.07%
11: -0.12%
12: -0.91%
13: 0.43%
14: -0.17%
15: 0.13%
16: -0.00%
17: 0.01%
18: 0.12%
19: -0.50%
20: 0.03%
21: 0.03%
22: 0.53%
23: 0.62%
24: -0.03%
25: -0.02%
26: 0.02%
27: 0.07%
28: 0.55%
29: 0.32%
30: 0.04%
31: -0.06%
32: -0.22%
33: 0.02%
34: 0.04%
35: 0.09%
36: -0.04%
37: 0.03%
38: -0.08%
39: -0.08%
40: -0.12%
41: 0.20%
42: -0.57%
43: 0.68%
44: 0.10%
45: 0.23%
46: 0.13%
47: -0.18%
48: -0.35%
49: 0.05%
50: -0.01%
51: 0.34%
52: 0.03%
53: -0.08%
54: 0.00%
55: 0.07%
56: 0.00%
57: -0.01%
58: 0.18%
59: -0.05%
60: 0.35%
61: 0.18%
62: -0.03%
63: 0.14%
64: -0.20%
65: 0.21%
66: -0.04%
67: -0.04%
68: -0.01%
69: -0.04%
70: 0.07%
71: 0.39%
72: -0.19%
73: -0.06%
74: 0.22%
75: 0.12%
76: 0.01%
77: 0.09%
78: 0.26%
79: 0.20%
80: -0.01%
81: 0.39%
82: -0.11%
83: -0.01%
84: 0.14%
85: -0.01%
86: 0.02%
87: 0.01%
88: -0.32%
89: -0.02%
90: -0.06%
91: 0.06%
92: -0.06%
93: 0.30%
9

In [14]:
pesos_finales = np.zeros(len(df_ultima_fecha['Rentabilidad_diaria'].values))
for ticker, peso in pesos_normalizados.items():
    pesos_finales[df_ultima_fecha.index.get_loc(ticker)] = peso

print(pesos_finales)

[ 0.00054916  0.00945945  0.00775666 ... -0.00047579  0.00133701
  0.00096897]


In [20]:
# Calcular la rentabilidad y el riesgo (desviación estándar) del portafolio
rentabilidad_total = np.dot(pesos_finales, df_ultima_fecha['Rentabilidad_diaria'].values)

print(f"\nRentabilidad Esperada de la Cartera: {rentabilidad_total:.2%}")



Rentabilidad Esperada de la Cartera: 19.33%


In [23]:
capital_total = 10000

df_ultima_fecha['Inversion'] = capital_total * df_ultima_fecha['rentabilidad_ponderada']
df_ultima_fecha

,Open_test,Ticker,Dates_test,Close_pred_test,Open_15d,Rentabilidad_diaria,rentabilidad_ponderada,Inversion
0,62.860001,AADR,2024-07-09,62.992689,62.630001,0.005791,0.000004,0.038996
1,29.610001,AAPB,2024-07-09,28.857473,26.240000,0.099751,0.000067,0.671726
2,16.860001,AAPD,2024-07-09,19.411267,17.943572,0.081795,0.000055,0.550808
3,27.500000,AAPY,2024-07-09,27.133608,27.000000,0.004948,0.000003,0.033323
4,73.849998,AAXJ,2024-07-09,74.063437,71.669998,0.033395,0.000022,0.224884
...,...,...,...,...,...,...,...,...
1480,8.530000,YANG,2024-07-09,8.655882,8.600000,0.006498,0.000004,0.043757
1481,3.260000,YOLO,2024-07-09,3.154076,3.500000,-0.098835,-0.000067,-0.665559
1482,17.219999,YXI,2024-07-09,17.213201,17.299999,-0.005017,-0.000003,-0.033786
1483,12.010000,YYY,2024-07-09,12.098302,11.930100,0.014099,0.000009,0.094943


In [ ]:
# Optimizar pesos para minimizar la varianza
resultado = minimize(funcion_objetivo, pesos_iniciales, constraints=restricciones, bounds=[(0, 1)] * len(tickers))



print(pesos_optimos)

# Mostrar resultados
print("\nPesos Óptimos para cada ETF:")
for ticker, peso in zip(tickers, pesos_optimos):
    print(f"{ticker}: {peso:.2%}")

# Calcular la rentabilidad y el riesgo (desviación estándar) del portafolio
rentabilidad_total = calcular_rentabilidad(pesos_iniciales, rentabilidad_esperada.values)
riesgo_total = np.sqrt(calcular_varianza(pesos_iniciales, rentabilidades_diarias_matrix))

print(f"\nRentabilidad Total del Portafolio: {rentabilidad_total:.2%}")
print(f"Riesgo Total del Portafolio (Desviación Estándar): {riesgo_total:.2%}")
print(f"Número de ETFs en los que se invierte: {np.sum(pesos_optimos > 0)}")


In [ ]:
# Función para calcular la rentabilidad esperada de la cartera
def calcular_rentabilidad(pesos, rentabilidades):
    return np.dot(pesos, rentabilidades)

# Función para calcular la varianza de la cartera
def calcular_varianza(pesos, cov_matrix):
    return np.dot(pesos.T, np.dot(cov_matrix, pesos))

# Restricciones
restricciones = [
    {'type': 'eq', 'fun': lambda pesos: np.sum(pesos) - 1},  # Suma de pesos debe ser 1
    {'type': 'ineq', 'fun': lambda pesos: pesos}  # Pesos >= 0
]

# Función de optimización para minimizar la varianza
def funcion_objetivo(pesos):
    return calcular_varianza(pesos, cov_matrix_top)

# Inicializar pesos uniformemente
pesos_iniciales = np.ones(len(tickers_top)) / len(tickers_top)

# Optimizar pesos para minimizar la varianza
resultado = minimize(funcion_objetivo, pesos_iniciales, constraints=restricciones, bounds=[(0, 1)] * len(tickers_top))

# Pesos óptimos
pesos_optimos = resultado.x

# Mostrar resultados
print("\nPesos Óptimos para cada ETF:")
for ticker, peso in zip(tickers_top, pesos_optimos):
    print(f"{ticker}: {peso:.2%}")

# Calcular la rentabilidad y el riesgo (desviación estándar) del portafolio
rentabilidad_total = calcular_rentabilidad(pesos_optimos, rentabilidad_esperada.values)
riesgo_total = np.sqrt(calcular_varianza(pesos_optimos, cov_matrix_top))

print(f"\nRentabilidad Total del Portafolio: {rentabilidad_total:.2%}")
print(f"Riesgo Total del Portafolio (Desviación Estándar): {riesgo_total:.2%}")
print(f"Número de ETFs en los que se invierte: {np.sum(pesos_optimos > 0)}")